In [47]:
from logic import *
from agents import *

class WumpusKB(PropKB):
    """
    Create a Knowledge Base that contains the a temporal "Wumpus physics" and temporal rules with time zero.
    """

    def __init__(self, dimrow):
        super().__init__()
        self.dimrow = dimrow
        self.tell(~wumpus(1, 1))
        self.tell(~pit(1, 1))

        for y in range(1, dimrow + 1):
            for x in range(1, dimrow + 1):

                pits_in = list()
                wumpus_in = list()

                if x > 1:  # West room exists
                    pits_in.append(pit(x - 1, y))
                    wumpus_in.append(wumpus(x - 1, y))

                if y < dimrow:  # North room exists
                    pits_in.append(pit(x, y + 1))
                    wumpus_in.append(wumpus(x, y + 1))

                if x < dimrow:  # East room exists
                    pits_in.append(pit(x + 1, y))
                    wumpus_in.append(wumpus(x + 1, y))

                if y > 1:  # South room exists
                    pits_in.append(pit(x, y - 1))
                    wumpus_in.append(wumpus(x, y - 1))

                self.tell(equiv(breeze(x, y), new_disjunction(pits_in)))
                self.tell(equiv(stench(x, y), new_disjunction(wumpus_in)))

        # Rule that describes existence of at least one Wumpus
        wumpus_at_least = list()
        for x in range(1, dimrow + 1):
            for y in range(1, dimrow + 1):
                wumpus_at_least.append(wumpus(x, y))

        self.tell(new_disjunction(wumpus_at_least))

        # Rule that describes existence of at most one Wumpus
        for i in range(1, dimrow + 1):
            for j in range(1, dimrow + 1):
                for u in range(1, dimrow + 1):
                    for v in range(1, dimrow + 1):
                        if i != u or j != v:
                            self.tell(~wumpus(i, j) | ~wumpus(u, v))

        # Temporal rules at time zero
        self.tell(location(1, 1, 0))
        for i in range(1, dimrow + 1):
            for j in range(1, dimrow + 1):
                self.tell(implies(location(i, j, 0), equiv(percept_breeze(0), breeze(i, j))))
                self.tell(implies(location(i, j, 0), equiv(percept_stench(0), stench(i, j))))
                if i != 1 or j != 1:
                    self.tell(~location(i, j, 0))

        self.tell(wumpus_alive(0))
        self.tell(have_arrow(0))
        self.tell(facing_east(0))
        self.tell(~facing_north(0))
        self.tell(~facing_south(0))
        self.tell(~facing_west(0))

    def make_action_sentence(self, action, time):
        actions = [move_forward(time), shoot(time), turn_left(time), turn_right(time)]
        print(action)
        for a in actions:
            if action == a:
                self.tell(action)
            else:
                self.tell(~a)

    def make_percept_sentence(self, percept, time):
        # Glitter, Bump, Stench, Breeze, Scream
        flags = [0, 0, 0, 0, 0]

        # Things perceived
        if isinstance(percept, Glitter):
            flags[0] = 1
            self.tell(percept_glitter(time))
        elif isinstance(percept, Bump):
            flags[1] = 1
            self.tell(percept_bump(time))
        elif isinstance(percept, Stench):
            flags[2] = 1
            self.tell(percept_stench(time))
        elif isinstance(percept, Breeze):
            flags[3] = 1
            self.tell(percept_breeze(time))
        elif isinstance(percept, Scream):
            flags[4] = 1
            self.tell(percept_scream(time))

        # Things not perceived
        for i in range(len(flags)):
            if flags[i] == 0:
                if i == 0:
                    self.tell(~percept_glitter(time))
                elif i == 1:
                    self.tell(~percept_bump(time))
                elif i == 2:
                    self.tell(~percept_stench(time))
                elif i == 3:
                    self.tell(~percept_breeze(time))
                elif i == 4:
                    self.tell(~percept_scream(time))

    def add_temporal_sentences(self, time):
        if time == 0:
            return
        t = time - 1

        # current location rules
        for i in range(1, self.dimrow + 1):
            for j in range(1, self.dimrow + 1):
                self.tell(implies(location(i, j, time), equiv(percept_breeze(time), breeze(i, j))))
                self.tell(implies(location(i, j, time), equiv(percept_stench(time), stench(i, j))))
                s = list()
                s.append(equiv(location(i, j, time), location(i, j, time) & ~move_forward(time) | percept_bump(time)))
                if i != 1:
                    s.append(location(i - 1, j, t) & facing_east(t) & move_forward(t))
                if i != self.dimrow:
                    s.append(location(i + 1, j, t) & facing_west(t) & move_forward(t))
                if j != 1:
                    s.append(location(i, j - 1, t) & facing_north(t) & move_forward(t))
                if j != self.dimrow:
                    s.append(location(i, j + 1, t) & facing_south(t) & move_forward(t))

                # add sentence about location i,j
                self.tell(new_disjunction(s))

                # add sentence about safety of location i,j
                self.tell(equiv(ok_to_move(i, j, time), ~pit(i, j) & ~wumpus(i, j) & wumpus_alive(time)))

        # Rules about current orientation

        a = facing_north(t) & turn_right(t)
        b = facing_south(t) & turn_left(t)
        c = facing_east(t) & ~turn_left(t) & ~turn_right(t)
        s = equiv(facing_east(time), a | b | c)
        self.tell(s)

        a = facing_north(t) & turn_left(t)
        b = facing_south(t) & turn_right(t)
        c = facing_west(t) & ~turn_left(t) & ~turn_right(t)
        s = equiv(facing_west(time), a | b | c)
        self.tell(s)

        a = facing_east(t) & turn_left(t)
        b = facing_west(t) & turn_right(t)
        c = facing_north(t) & ~turn_left(t) & ~turn_right(t)
        s = equiv(facing_north(time), a | b | c)
        self.tell(s)

        a = facing_west(t) & turn_left(t)
        b = facing_east(t) & turn_right(t)
        c = facing_south(t) & ~turn_left(t) & ~turn_right(t)
        s = equiv(facing_south(time), a | b | c)
        self.tell(s)

        # Rules about last action
        self.tell(equiv(move_forward(t), ~turn_right(t) & ~turn_left(t)))

        # Rule about the arrow
        self.tell(equiv(have_arrow(time), have_arrow(t) & ~shoot(t)))

        # Rule about Wumpus (dead or alive)
        self.tell(equiv(wumpus_alive(time), wumpus_alive(t) & ~percept_scream(time)))

    def ask_if_true(self, query):
        return pl_resolution(self, query)


wkb = WumpusKB(3)

In [48]:
wkb.clauses

[~W(1, 1),
 ~P(1, 1),
 (~P(1, 2) | B(1, 1)),
 (~P(2, 1) | B(1, 1)),
 (P(1, 2) | P(2, 1) | ~B(1, 1)),
 (~W(1, 2) | S(1, 1)),
 (~W(2, 1) | S(1, 1)),
 (W(1, 2) | W(2, 1) | ~S(1, 1)),
 (~P(1, 1) | B(2, 1)),
 (~P(2, 2) | B(2, 1)),
 (~P(3, 1) | B(2, 1)),
 (P(1, 1) | P(2, 2) | P(3, 1) | ~B(2, 1)),
 (~W(1, 1) | S(2, 1)),
 (~W(2, 2) | S(2, 1)),
 (~W(3, 1) | S(2, 1)),
 (W(1, 1) | W(2, 2) | W(3, 1) | ~S(2, 1)),
 (~P(2, 1) | B(3, 1)),
 (~P(3, 2) | B(3, 1)),
 (P(2, 1) | P(3, 2) | ~B(3, 1)),
 (~W(2, 1) | S(3, 1)),
 (~W(3, 2) | S(3, 1)),
 (W(2, 1) | W(3, 2) | ~S(3, 1)),
 (~P(1, 3) | B(1, 2)),
 (~P(2, 2) | B(1, 2)),
 (~P(1, 1) | B(1, 2)),
 (P(1, 3) | P(2, 2) | P(1, 1) | ~B(1, 2)),
 (~W(1, 3) | S(1, 2)),
 (~W(2, 2) | S(1, 2)),
 (~W(1, 1) | S(1, 2)),
 (W(1, 3) | W(2, 2) | W(1, 1) | ~S(1, 2)),
 (~P(1, 2) | B(2, 2)),
 (~P(2, 3) | B(2, 2)),
 (~P(3, 2) | B(2, 2)),
 (~P(2, 1) | B(2, 2)),
 (P(1, 2) | P(2, 3) | P(3, 2) | P(2, 1) | ~B(2, 2)),
 (~W(1, 2) | S(2, 2)),
 (~W(2, 3) | S(2, 2)),
 (~W(3, 2) | S(2, 2)),


In [49]:
wkb.make_action_sentence(move_forward(0), 0)

Forward(0)


In [50]:
wkb.clauses

[~W(1, 1),
 ~P(1, 1),
 (~P(1, 2) | B(1, 1)),
 (~P(2, 1) | B(1, 1)),
 (P(1, 2) | P(2, 1) | ~B(1, 1)),
 (~W(1, 2) | S(1, 1)),
 (~W(2, 1) | S(1, 1)),
 (W(1, 2) | W(2, 1) | ~S(1, 1)),
 (~P(1, 1) | B(2, 1)),
 (~P(2, 2) | B(2, 1)),
 (~P(3, 1) | B(2, 1)),
 (P(1, 1) | P(2, 2) | P(3, 1) | ~B(2, 1)),
 (~W(1, 1) | S(2, 1)),
 (~W(2, 2) | S(2, 1)),
 (~W(3, 1) | S(2, 1)),
 (W(1, 1) | W(2, 2) | W(3, 1) | ~S(2, 1)),
 (~P(2, 1) | B(3, 1)),
 (~P(3, 2) | B(3, 1)),
 (P(2, 1) | P(3, 2) | ~B(3, 1)),
 (~W(2, 1) | S(3, 1)),
 (~W(3, 2) | S(3, 1)),
 (W(2, 1) | W(3, 2) | ~S(3, 1)),
 (~P(1, 3) | B(1, 2)),
 (~P(2, 2) | B(1, 2)),
 (~P(1, 1) | B(1, 2)),
 (P(1, 3) | P(2, 2) | P(1, 1) | ~B(1, 2)),
 (~W(1, 3) | S(1, 2)),
 (~W(2, 2) | S(1, 2)),
 (~W(1, 1) | S(1, 2)),
 (W(1, 3) | W(2, 2) | W(1, 1) | ~S(1, 2)),
 (~P(1, 2) | B(2, 2)),
 (~P(2, 3) | B(2, 2)),
 (~P(3, 2) | B(2, 2)),
 (~P(2, 1) | B(2, 2)),
 (P(1, 2) | P(2, 3) | P(3, 2) | P(2, 1) | ~B(2, 2)),
 (~W(1, 2) | S(2, 2)),
 (~W(2, 3) | S(2, 2)),
 (~W(3, 2) | S(2, 2)),


In [53]:
wkb.add_temporal_sentences(1)
wkb.make_action_sentence(turn_left(1), 1)

TurnLeft(1)


In [58]:
wkb.ask_if_true(facing_east(0))

True

In [59]:
wkb.ask_if_true(facing_east(1))

KeyboardInterrupt: 